In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import obspy
from collections import defaultdict
from obspy.clients.fdsn import Client
from datetime import datetime

In [ ]:
config = {}
# config["starttime"] = catalog["time"].min().isoformat()
# config["endtime"] = catalog["time"].max().isoformat()
config["starttime"] = "2022-06-03T00:00:00.000"
config["endtime"] = "2022-06-17T00:00:00.000"
config["xlim_degree"] = [-155.32-1, -155.32+1]
config["ylim_degree"] = [19.39-1, 19.39+1]

bins = len(pd.date_range(datetime.fromisoformat(config["starttime"]), datetime.fromisoformat(config["endtime"]), freq="1H"))

In [ ]:

events = Client("iris").get_events(
    starttime=config["starttime"],
    endtime=config["endtime"],
    minlongitude=config["xlim_degree"][0],
    maxlongitude=config["xlim_degree"][1],
    minlatitude=config["ylim_degree"][0],
    maxlatitude=config["ylim_degree"][1],
    # filename='events.xml',
)

#     events = obspy.read_events('events.xml')
print(f"Number of events: {len(events)}")
#     events.plot('local', outfile="events.png")
#     events.plot('local')

####### Save catalog ########
catalog = defaultdict(list)
for event in events:
    if len(event.magnitudes) > 0:
        catalog["time"].append(event.origins[0].time.datetime)
        catalog["magnitude"].append(event.magnitudes[0].mag)
        catalog["longitude"].append(event.origins[0].longitude)
        catalog["latitude"].append(event.origins[0].latitude)
        catalog["depth(m)"].append(event.origins[0].depth)
catalog = pd.DataFrame.from_dict(catalog).sort_values(["time"])
catalog.to_csv(
    "catalog_iris.csv",
    # sep="\t",
    index=False,
    float_format="%.3f",
    date_format="%Y-%m-%dT%H:%M:%S.%f",
    columns=["time", "magnitude", "longitude", "latitude", "depth(m)"],
)

In [ ]:
iris_catalog = pd.read_csv("catalog_iris.csv")

In [ ]:
das_info = pd.read_csv("das_info.csv")

In [ ]:
# plt.figure()
# plt.scatter(iris_catalog["longitude"], iris_catalog["latitude"], s=iris_catalog["magnitude"] * 10)

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from matplotlib.offsetbox import AnchoredText


fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.set_extent([config["xlim_degree"][0]+0.3, config["xlim_degree"][1]-0.3, config["ylim_degree"][0]+0.3, config["ylim_degree"][1]-0.3], crs=ccrs.PlateCarree())

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.STATES, linestyle='--')
gl = ax.gridlines(draw_labels=True, dms=False, x_inline=False, y_inline=False)
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlines = False
gl.ylines = False

## add anchor text to the mean of das_info
im = ax.scatter(iris_catalog["longitude"], iris_catalog["latitude"], s=iris_catalog["magnitude"] * 10, c=iris_catalog["depth(m)"]/1e3, cmap="viridis_r", marker="x", transform=ccrs.PlateCarree(), label="IRIS catalog")
fig.colorbar(im, ax=ax, label="Depth (km)")
im = ax.scatter(das_info["longitude"], das_info["latitude"], s=10, c="r", transform=ccrs.PlateCarree(), label="DAS location")
plt.legend()

## add zoomin view of the das_info
axins = ax.inset_axes([0.7, 0.7, 0.3, 0.3])
axins.scatter(das_info["longitude"], das_info["latitude"], s=1, c="r", label="DAS")
x1 = das_info["longitude"].min() - 0.005
x2 = das_info["longitude"].max() + 0.005
y1 = das_info["latitude"].min() - 0.005
y2 = das_info["latitude"].max() + 0.005
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticks([])
axins.set_yticks([])

ax.indicate_inset_zoom(axins, edgecolor="black")

plt.title("HVO Earthquake Catalog")
plt.savefig("iris_catalog.png", dpi=300)
plt.show()

In [ ]:
catalog = pd.read_csv("catalog_gamma_.csv", parse_dates=["time"])

In [ ]:
plt.figure(figsize=(10, 3))
idx = (catalog["number_p_picks"] > 100)
plt.hist(catalog[idx]["time"], bins=bins//5, range=(config["starttime"], config["endtime"]), edgecolor="white", linewidth=1, label="DAS")
plt.hist(iris_catalog["time"], bins=bins//5, range=(config["starttime"], config["endtime"]), edgecolor="white", linewidth=1, alpha=0.8, label="IRIS")
# plt.gcf().autofmt_xdate()
plt.ylabel("Fequency")
plt.xlabel("Date")
plt.legend()
# set xrange tight
plt.xlim(pd.to_datetime(config["starttime"]), pd.to_datetime(config["endtime"]))
plt.savefig("gamma_time_.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.set_extent([config["xlim_degree"][0]+0.3, config["xlim_degree"][1]-0.3, config["ylim_degree"][0]+0.3, config["ylim_degree"][1]-0.3], crs=ccrs.PlateCarree())

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.STATES, linestyle='--')
gl = ax.gridlines(draw_labels=True, dms=False, x_inline=False, y_inline=False)
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlines = False
gl.ylines = False

## add anchor text to the mean of das_info
im = ax.scatter(catalog[idx]["longitude"], catalog[idx]["latitude"], s=2.0, c="k", cmap="viridis_r", marker="x", transform=ccrs.PlateCarree(), label="Event")
fig.colorbar(im, ax=ax, label="Depth (km)")

im = ax.scatter(das_info["longitude"], das_info["latitude"], s=10, c="r", transform=ccrs.PlateCarree(), label="DAS")

# plt.legend()
## add zoomin view of the das_info
axins = ax.inset_axes([0.7, 0.7, 0.3, 0.3])
axins.scatter(das_info["longitude"], das_info["latitude"], s=1, c="r", label="DAS")
x1 = das_info["longitude"].min() - 0.005
x2 = das_info["longitude"].max() + 0.005
y1 = das_info["latitude"].min() - 0.005
y2 = das_info["latitude"].max() + 0.005
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticklabels([])
axins.set_yticklabels([])

ax.indicate_inset_zoom(axins, edgecolor="black")

plt.title("DAS Earthquake Catalog")
plt.savefig("gamma_catalog_.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
catalog = pd.read_csv("catalog_gamma_5Hz.csv", parse_dates=["time"])

In [ ]:
plt.figure(figsize=(10, 3))
idx = (catalog["number_p_picks"] > 100)
plt.hist(catalog[idx]["time"], bins=bins//5, range=(config["starttime"], config["endtime"]), edgecolor="white", linewidth=1, label="DAS")
plt.hist(iris_catalog["time"], bins=bins//5, range=(config["starttime"], config["endtime"]), edgecolor="white", linewidth=1, alpha=0.8, label="IRIS")
# plt.gcf().autofmt_xdate()
plt.ylabel("Fequency")
plt.xlabel("Date")
plt.legend()
# set xrange tight
plt.xlim(pd.to_datetime(config["starttime"]), pd.to_datetime(config["endtime"]))
plt.savefig("gamma_time_5Hz.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.set_extent([config["xlim_degree"][0]+0.3, config["xlim_degree"][1]-0.3, config["ylim_degree"][0]+0.3, config["ylim_degree"][1]-0.3], crs=ccrs.PlateCarree())

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.STATES, linestyle='--')
gl = ax.gridlines(draw_labels=True, dms=False, x_inline=False, y_inline=False)
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlines = False
gl.ylines = False

## add anchor text to the mean of das_info
im = ax.scatter(catalog[idx]["longitude"], catalog[idx]["latitude"], s=2.0, c="k", cmap="viridis_r", marker="x", transform=ccrs.PlateCarree(), label="Event")
fig.colorbar(im, ax=ax, label="Depth (km)")

im = ax.scatter(das_info["longitude"], das_info["latitude"], s=10, c="r", transform=ccrs.PlateCarree(), label="DAS")

# plt.legend()
## add zoomin view of the das_info
axins = ax.inset_axes([0.7, 0.7, 0.3, 0.3])
axins.scatter(das_info["longitude"], das_info["latitude"], s=1, c="r", label="DAS")
x1 = das_info["longitude"].min() - 0.005
x2 = das_info["longitude"].max() + 0.005
y1 = das_info["latitude"].min() - 0.005
y2 = das_info["latitude"].max() + 0.005
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticklabels([])
axins.set_yticklabels([])

ax.indicate_inset_zoom(axins, edgecolor="black")

plt.title("DAS Earthquake Catalog")
plt.savefig("gamma_catalog_5Hz.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
catalog = pd.read_csv("catalog_gamma_10Hz.csv", parse_dates=["time"])

In [ ]:
plt.figure(figsize=(10, 3))
idx = (catalog["number_p_picks"] > 100)
plt.hist(catalog[idx]["time"], bins=bins//5, range=(config["starttime"], config["endtime"]), edgecolor="white", linewidth=1, label="DAS")
plt.hist(iris_catalog["time"], bins=bins//5, range=(config["starttime"], config["endtime"]), edgecolor="white", linewidth=1, alpha=0.8, label="IRIS")
# plt.gcf().autofmt_xdate()
plt.ylabel("Fequency")
plt.xlabel("Date")
plt.legend()
# set xrange tight
plt.xlim(pd.to_datetime(config["starttime"]), pd.to_datetime(config["endtime"]))
plt.savefig("gamma_time_10Hz.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.set_extent([config["xlim_degree"][0]+0.3, config["xlim_degree"][1]-0.3, config["ylim_degree"][0]+0.3, config["ylim_degree"][1]-0.3], crs=ccrs.PlateCarree())

ax.add_feature(cfeature.LAND)
ax.add_feature(cfeature.OCEAN)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.add_feature(cfeature.STATES, linestyle='--')

gl = ax.gridlines(draw_labels=True, dms=False, x_inline=False, y_inline=False)
gl.xlabels_top = False
gl.ylabels_right = False
gl.xlines = False
gl.ylines = False

## add anchor text to the mean of das_info
im = ax.scatter(catalog[idx]["longitude"], catalog[idx]["latitude"], s=2.0, c="k", cmap="viridis_r", marker="x", transform=ccrs.PlateCarree(), label="Event")
fig.colorbar(im, ax=ax, label="Depth (km)")

im = ax.scatter(das_info["longitude"], das_info["latitude"], s=10, c="r", transform=ccrs.PlateCarree(), label="DAS")

# plt.legend()
## add zoomin view of the das_info
axins = ax.inset_axes([0.7, 0.7, 0.3, 0.3])
axins.scatter(das_info["longitude"], das_info["latitude"], s=1, c="r", label="DAS")
x1 = das_info["longitude"].min() - 0.005
x2 = das_info["longitude"].max() + 0.005
y1 = das_info["latitude"].min() - 0.005
y2 = das_info["latitude"].max() + 0.005
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticklabels([])
axins.set_yticklabels([])

ax.indicate_inset_zoom(axins, edgecolor="black")

plt.title("DAS Earthquake Catalog")
plt.savefig("gamma_catalog_10Hz.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
pick_path = Path("../../Hawaii_debug2/picks_phasenet_das_raw/")
pick_files = list(pick_path.glob("*.csv"))[:500]
picks = []
for pick_file in tqdm(pick_files):
    try:
        pick_ = pd.read_csv(pick_file, parse_dates=["phase_time"])
        pick_ = pick_[pick_["phase_index"] < 178500]
        picks.append(pick_)
    except:
        pass
picks = pd.concat(picks)
picks.to_csv("picks_.csv", index=False)

In [ ]:
# picks = pd.read_csv("picks_.csv", parse_dates=["phase_time"])

In [ ]:
bins = len(pd.date_range(picks["phase_time"].min(), picks["phase_time"].max(), freq="1H"))
colors = picks["phase_type"].map({"P": "red", "S": "blue"})
idx = (picks["phase_type"] == "P")
plt.figure(figsize=(bins, 5))
# plt.hist(picks["phase_time"], bins=bins)
plt.scatter(picks[idx]["phase_time"], picks[idx]["channel_index"], s=0.1, c=colors[idx], alpha=0.5)
plt.gcf().autofmt_xdate()
plt.savefig("picks_.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
pick_path = Path("../../Hawaii/picks_phasenet_das_raw/")
pick_files = list(pick_path.glob("*.csv"))[:500]
picks = []
for pick_file in tqdm(pick_files):
    try:
        pick_ = pd.read_csv(pick_file, parse_dates=["phase_time"])
        pick_ = pick_[pick_["phase_index"] < 178500]
        picks.append(pick_)
    except:
        pass
picks = pd.concat(picks)
picks.to_csv("picks_.csv", index=False)

In [ ]:
bins = len(pd.date_range(picks["phase_time"].min(), picks["phase_time"].max(), freq="1H"))
colors = picks["phase_type"].map({"P": "red", "S": "blue"})
idx = (picks["phase_type"] == "P")
plt.figure(figsize=(bins, 5))
# plt.hist(picks["phase_time"], bins=bins)
plt.scatter(picks[idx]["phase_time"], picks[idx]["channel_index"], s=0.1, c=colors[idx], alpha=0.5)
plt.gcf().autofmt_xdate()
plt.savefig("picks_.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
pick_path = Path("../../Hawaii_debug2/picks_phasenet_das_raw/")
pick_files = list(pick_path.glob("*.csv"))[:500]
picks = []
for pick_file in tqdm(pick_files):
    try:
        pick_ = pd.read_csv(pick_file, parse_dates=["phase_time"])
        # pick_ = pick_[pick_["phase_index"] < 178500]
        picks.append(pick_)
    except Exception as e:
        print(e)
        pass
picks = pd.concat(picks)
picks.to_csv("picks_.csv", index=False)

In [ ]:
bins = len(pd.date_range(picks["phase_time"].min(), picks["phase_time"].max(), freq="1H"))
colors = picks["phase_type"].map({"P": "red", "S": "blue"})
idx = (picks["phase_type"] == "P")
plt.figure(figsize=(bins, 5))
# plt.hist(picks["phase_time"], bins=bins)
plt.scatter(picks[idx]["phase_time"], picks[idx]["channel_index"], s=0.1, c=colors[idx], alpha=0.5)
plt.gcf().autofmt_xdate()
plt.savefig("picks_debug.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
bins = len(pd.date_range(picks["phase_time"].min(), picks["phase_time"].max(), freq="1H"))
colors = picks["phase_type"].map({"P": "red", "S": "blue"})
# plt.figure(figsize=(bins/10, 5))
plt.figure(figsize=(10, 3))
idx = (picks["phase_type"] == "P")
plt.hist(picks[idx]["phase_time"], bins=bins//5, range=(config["starttime"], config["endtime"]), edgecolor="white", linewidth=1, label="P picks")
# idx = (picks["phase_type"] == "S")
# plt.hist(picks[idx]["phase_time"], bins=bins//5, range=(config["starttime"], config["endtime"]), edgecolor="white", linewidth=1, label="S picks")
plt.legend()
plt.ylabel("Fequency")
plt.xlim(pd.to_datetime(config["starttime"]), pd.to_datetime(config["endtime"]))
plt.savefig("picks_hist_.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# pick_path = Path("../../Hawaii_5Hz/picks_phasenet_das_raw/")
# pick_files = list(pick_path.glob("*.csv"))
# picks = []
# for pick_file in tqdm(pick_files):
#     try:
#         pick_ = pd.read_csv(pick_file, parse_dates=["phase_time"])
#         pick_ = pick_[pick_["phase_index"] < 178500]
#         picks.append(pick_)
#     except:
#         pass
# picks = pd.concat(picks)
# picks.to_csv("picks_5Hz.csv", index=False)


In [ ]:
picks = pd.read_csv("picks_5Hz.csv", parse_dates=["phase_time"])

In [ ]:
bins = len(pd.date_range(picks["phase_time"].min(), picks["phase_time"].max(), freq="1H"))
colors = picks["phase_type"].map({"P": "red", "S": "blue"})
idx = (picks["phase_type"] == "P")
plt.figure(figsize=(bins/10, 5))
plt.scatter(picks[idx]["phase_time"], picks[idx]["channel_index"], s=0.1, c=colors[idx], alpha=0.5)
plt.gcf().autofmt_xdate()
plt.ylabel("Channel Index")
plt.savefig("picks_5Hz.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
bins = len(pd.date_range(picks["phase_time"].min(), picks["phase_time"].max(), freq="1H"))
colors = picks["phase_type"].map({"P": "red", "S": "blue"})
# plt.figure(figsize=(bins/10, 5))
plt.figure(figsize=(10, 3))
idx = (picks["phase_type"] == "P")
plt.hist(picks[idx]["phase_time"], bins=bins//5, range=(config["starttime"], config["endtime"]), edgecolor="white", linewidth=1, label="P picks")
# idx = (picks["phase_type"] == "S")
# plt.hist(picks[idx]["phase_time"], bins=bins//5, range=(config["starttime"], config["endtime"]), edgecolor="white", linewidth=1, label="S picks")
plt.legend()
plt.ylabel("Fequency")
plt.xlim(pd.to_datetime(config["starttime"]), pd.to_datetime(config["endtime"]))
plt.savefig("picks_hist_5Hz.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# bins = len(pd.date_range(picks["phase_time"].min(), picks["phase_time"].max(), freq="1H"))
# colors = picks["phase_type"].map({"P": "red", "S": "blue"})
# idx = (picks["phase_type"] == "P")
# plt.figure(figsize=(bins/5, 5))
# plt.scatter(picks[idx]["phase_time"], picks[idx]["channel_index"], s=0.1, c=colors[idx], alpha=0.5)
# plt.gcf().autofmt_xdate()
# plt.show()

In [ ]:
# pick_path = Path("../../Hawaii_10Hz/picks_phasenet_das_raw/")
# pick_files = list(pick_path.glob("*.csv"))
# picks = []
# for i, pick_file in enumerate(tqdm(pick_files)):
#     try:
#         pick_ = pd.read_csv(pick_file, parse_dates=["phase_time"])
#         pick_ = pick_[pick_["phase_index"] < 178500]
#         picks.append(pick_)
#     except:
#         pass

# picks = pd.concat(picks)
# picks.to_csv("picks_10Hz.csv", index=False)

In [ ]:
picks = pd.read_csv("picks_10Hz.csv", parse_dates=["phase_time"])

In [ ]:
bins = len(pd.date_range(picks["phase_time"].min(), picks["phase_time"].max(), freq="1H"))
colors = picks["phase_type"].map({"P": "red", "S": "blue"})
idx = (picks["phase_type"] == "P")
plt.figure(figsize=(bins/10, 5))
plt.scatter(picks[idx]["phase_time"], picks[idx]["channel_index"], s=0.1, c=colors[idx], alpha=0.5)
plt.gcf().autofmt_xdate()
plt.ylabel("Channel Index")
plt.savefig("picks_10Hz.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
bins = len(pd.date_range(picks["phase_time"].min(), picks["phase_time"].max(), freq="1H"))
colors = picks["phase_type"].map({"P": "red", "S": "blue"})
# plt.figure(figsize=(bins/10, 5))
plt.figure(figsize=(10, 3))
idx = (picks["phase_type"] == "P")
plt.hist(picks[idx]["phase_time"], bins=bins//5, range=(config["starttime"], config["endtime"]), edgecolor="white", linewidth=1, label="P picks")
# idx = (picks["phase_type"] == "S")
# plt.hist(picks[idx]["phase_time"], bins=bins//5, range=(config["starttime"], config["endtime"]), edgecolor="white", linewidth=1, label="S picks")
plt.legend()
plt.ylabel("Fequency")
plt.xlim(pd.to_datetime(config["starttime"]), pd.to_datetime(config["endtime"]))
plt.savefig("picks_hist_10Hz.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
plt.figure()
plt.scatter(catalog[idx]["latitude"], catalog[idx]["longitude"], s=0.1, alpha=0.5)

In [ ]:
catalog["time"].min().isoformat()

In [ ]:
catalog["time"].max()

In [ ]:
plt.figure(figsize=(30, 5))
plt.hist(catalog["time"], bins=100)
plt.gcf().autofmt_xdate()
plt.ylabel("Fequency")
plt.savefig("iris_time.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
import h5py

In [ ]:
data_path = Path("/kuafu/DASdata/Hawaii_desampled/")
h5_files = list(data_path.glob("*.h5"))

In [ ]:
for h5_file in h5_files:
    print(h5_file)
    with h5py.File(h5_file, "r") as fp:
        print(fp["Data"].shape)
        raw_data = fp["Data"][:]

    raise

In [ ]:
data = raw_data.copy()

In [ ]:
# data = data.T
data = np.gradient(data, axis=-1)

In [ ]:
data = data - np.mean(data, axis=-1, keepdims=True)
data = data / np.std(data, axis=-1, keepdims=True)

In [ ]:
print(data.shape)

In [ ]:
plt.figure()
plt.imshow(data[:, 179500:].T, vmin=-1, vmax=1, aspect="auto", cmap="seismic")
plt.colorbar()
plt.show()